# KorEDA

## Load Library

In [30]:
import random
import pickle
import re

## Load WordNet

In [31]:
with open("wordnet.pickle", "rb") as f:
    wordnet = pickle.load(f)

In [32]:
wordnet

{'호흡하': ['호흡하'],
 '물': ['물', '놈'],
 '질식하': ['질식하'],
 '전체': ['전체', '전적', '전부', '총체', '전반'],
 '한숨짓': ['한숨짓'],
 '생물': ['생물'],
 '미생물': ['미생물', '생물'],
 '헐떡이': ['헐떡이'],
 '셀': ['셀', '세포'],
 '하품하': ['하품하'],
 '인': ['인', '인간', '인물', '사람', '인류'],
 '미물': ['미물', '생물', '조수'],
 '기상하': ['기상하'],
 '자연물': ['자연물'],
 '영양': ['영양', '자양', '영양분', '양분', '영양소'],
 '물건': ['물건', '기품', '물', '물품', '물체', '놈'],
 '학식': ['학식', '학문', '인식', '알음알이', '인지', '지식', '경험', '식견'],
 '속성': ['속성'],
 '긴장하': ['긴장하'],
 '감정': ['감정', '마음', '기분'],
 '유형': ['유형',
  '폼',
  '형상',
  '체형',
  '형',
  '자세',
  '맵시',
  '형태',
  '형용',
  '복장',
  '꼴',
  '거푸집',
  '모양',
  '짝',
  '형체',
  '용모',
  '외형',
  '모양새',
  '모형'],
 '시간': ['시간', '성상', '세월'],
 '길': ['길', '과정', '경로'],
 '짓': ['짓', '법', '소위', '행위'],
 '집단': ['집단', '무리', '군', '통'],
 '상관': ['상관', '연고', '관계', '인연', '사이', '동안', '연관', '관련'],
 '재': ['재', '소장', '것', '점령', '소유물', '손', '소유', '향유', '점유', '재물'],
 '커뮤니케이션': ['커뮤니케이션', '교신', '통신'],
 '수량': ['수량', '정량', '양', '분량', '숱'],
 '속임질하': ['속임질하'],
 '중단': ['중단'],
 '

## Method

In [33]:
sentence = "제가 우울감을 느낀지는 오래됐는데 점점 개선되고 있다고 느껴요"

In [34]:
def get_only_hangul(sentence):
    return ' '.join(re.findall(r'[\uac00-\ud7a3]+', sentence))


### SR(synonym replacement)

In [35]:
def get_synonyms(word: str) -> list:
    # 동의어 목록을 반환하거나 빈 리스트 반환
    return wordnet.get(word, [])

def synonym_replacement(words: list, n: int) -> list:
    new_words = words.copy()
    random.shuffle(new_words) # 단어 목록을 무작위로 섞음
    num_replaced = 0
    
    for idx in range(len(new_words)):
        synonyms = get_synonyms(new_words[idx])
        if synonyms: # 동의어가 있는 경우
            synonym = random.choice(synonyms) # 무작위 동의어 선택
            new_words[idx] = synonym # 단어 교체
            num_replaced += 1
        if num_replaced >= n: # 교체할 단어 수에 도달한 경우 반복 종료
            break
        
    return new_words

n = 5
print(synonym_replacement(sentence.split(), n))

['느낀지는', '제가', '우울감을', '개선되고', '점점', '느껴요', '있다고', '오래됐는데']


### RD(random deletion)

In [36]:
def random_deletion(words: list, p: float) -> list:
    # 리스트에 단어가 하나만 있으면 그대로 반환
    if len(words) == 1:
        return words
    
    new_words = [word for word in words if random.random() > p] # 확률 p보다 큰 경우에만 단어 유지
    
    # 모든 단어가 삭제된 경우 하나의 단어를 임의로 선택하여 반환
    return new_words if new_words else [random.choice(words)]

# 예제 실행
p = 0.3 
print(random_deletion(sentence.split(), p))

['제가', '오래됐는데', '점점', '개선되고', '느껴요']


### RS(random swap)

In [37]:
def random_swap(words: list, n: int) -> list:
    new_words = words.copy()
    length = len(new_words)
    for _ in range(n):
        if length > 1: # 최소 2개의 단어가 있는지 확인
            random_idx_1 = random.randint(0, length - 1)
            random_idx_2 = random_idx_1
            while random_idx_2 == random_idx_1:
                random_idx_2 = random.randint(0, length - 1)
            new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
            
    return new_words

# Example usage:
n = 3 # Number of swaps
print(random_swap(sentence.split(), n))

['개선되고', '우울감을', '점점', '느낀지는', '오래됐는데', '제가', '있다고', '느껴요']


### RI(random insertion)

In [38]:
def random_insertion(words: list, n: int, get_synonyms: get_synonyms) -> list:
    new_words = words.copy()
    length = len(new_words)
    
    for _ in range(n):
        if length == 0: # 동의어가 없으면 무시
            break
        
        # 무작위 선택
        random_word = new_words[random.randint(0, length - 1)]
        synonyms = get_synonyms(random_word)
        
        if synonyms:
            random_synonym = random.choice(synonyms)
            random_idx = random.randint(0, length)
            new_words.insert(random_idx, random_synonym)
            length += 1 
            
    return new_words

n = 3
print(random_insertion(sentence.split(), n, get_synonyms))

['제가', '우울감을', '느낀지는', '오래됐는데', '점점', '개선되고', '있다고', '느껴요']


## EDA

In [39]:
def EDA(sentence: str, alpha_sr: float=0.1, alpha_ri: float=0.1, alpha_rs: float=0.1,
        p_rd: float=0.1, num_aug: int=9) -> list:
    sentence = get_only_hangul(sentence)
    words = sentence.split()
    num_words = len(words)
    num_new_per_technique = int(num_aug / 4) + 1
    
    # Calculate number of words to modify based on the length of the sentence
    n_sr = max(1, int(alpha_sr * num_words))
    n_ri = max(1, int(alpha_ri * num_words))
    n_rs = max(1, int(alpha_rs * num_words))
    
    augmented_sentences = []
    
    # Perform each augmentation technique
    for _ in range(num_new_per_technique):
        augmented_sentences.append(' '.join(synonym_replacement(words, n_sr)))
        augmented_sentences.append(' '.join(random_insertion(words, n_ri, get_synonyms)))
        augmented_sentences.append(' '.join(random_swap(words, n_rs)))
        augmented_sentences.append(' '.join(random_deletion(words, p_rd)))
        
    # print(augmented_sentences)
    # Ensure only valid Hangul remains, shuffle, and trim to the desired number of augmented sentences
    augmented_sentences = [get_only_hangul(s) for s in augmented_sentences if s.strip()]
    random.shuffle(augmented_sentences)
    augmented_sentences = augmented_sentences[:num_aug]
    
    # Always include the original sentence
    augmented_sentences.append(sentence)
    
    return augmented_sentences

In [40]:
tmp = EDA(sentence)

tmp

['우울감을 있다고 느낀지는 제가 느껴요 개선되고 점점 오래됐는데',
 '제가 우울감을 느낀지는 오래됐는데 개선되고 있다고 느껴요',
 '제가 우울감을 느낀지는 오래됐는데 점점 개선되고 있다고 느껴요',
 '제가 우울감을 느낀지는 점점 오래됐는데 개선되고 있다고 느껴요',
 '제가 우울감을 느낀지는 오래됐는데 점점 개선되고 있다고 느껴요',
 '제가 우울감을 느낀지는 오래됐는데 개선되고 점점 있다고 느껴요',
 '개선되고 느껴요 우울감을 제가 점점 느낀지는 있다고 오래됐는데',
 '제가 우울감을 느낀지는 오래됐는데 점점 개선되고 있다고 느껴요',
 '우울감을 제가 오래됐는데 있다고 느낀지는 개선되고 점점 느껴요',
 '제가 우울감을 느낀지는 오래됐는데 점점 개선되고 있다고 느껴요']

# Sentence Transformers

한국어 sentence transformers 모델을 가져와 학습을 시킨 후, 데이터를 증강하는 방법 추천

In [41]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
from sentence_transformers import SentenceTransformer, util
import numpy as np

embedder = SentenceTransformer("jhgan/ko-sroberta-multitask")

# Corpus with example sentences
corpus = ['한 남자가 음식을 먹는다.',
 '한 남자가 빵 한 조각을 먹는다.',
 '그 여자가 아이를 돌본다.',
 '한 남자가 말을 탄다.',
 '한 여자가 바이올린을 연주한다.',
 '두 남자가 수레를 숲 속으로 밀었다.',
 '한 남자가 담으로 싸인 땅에서 백마를 타고 있다.',
 '원숭이 한 마리가 드럼을 연주한다.',
 '치타 한 마리가 먹이 뒤에서 달리고 있다.']

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['한 남자가 파스타를 먹는다.',
  '고릴라 의상을 입은 누군가가 드럼을 연주하고 있다.',
  '치타가 들판을 가로 질러 먹이를 쫓는다.']

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
for query in queries:
 query_embedding = embedder.encode(query, convert_to_tensor=True)
 cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
 cos_scores = cos_scores.cpu()

 #We use np.argpartition, to only partially sort the top_k results
 top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

 print("\n\n======================\n\n")
 print("Query:", query)
 print("\nTop 5 most similar sentences in corpus:")

 for idx in top_results[0:top_k]:
  print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))
embedder = SentenceTransformer()





Query: 한 남자가 파스타를 먹는다.

Top 5 most similar sentences in corpus:
한 남자가 음식을 먹는다. (Score: 0.5742)
한 남자가 빵 한 조각을 먹는다. (Score: 0.4833)
한 남자가 말을 탄다. (Score: 0.1337)
치타 한 마리가 먹이 뒤에서 달리고 있다. (Score: 0.1249)
한 남자가 담으로 싸인 땅에서 백마를 타고 있다. (Score: 0.1028)




Query: 고릴라 의상을 입은 누군가가 드럼을 연주하고 있다.

Top 5 most similar sentences in corpus:
원숭이 한 마리가 드럼을 연주한다. (Score: 0.6619)
한 여자가 바이올린을 연주한다. (Score: 0.2921)
치타 한 마리가 먹이 뒤에서 달리고 있다. (Score: 0.2271)
한 남자가 담으로 싸인 땅에서 백마를 타고 있다. (Score: 0.1968)
한 남자가 음식을 먹는다. (Score: 0.1248)




Query: 치타가 들판을 가로 질러 먹이를 쫓는다.

Top 5 most similar sentences in corpus:
치타 한 마리가 먹이 뒤에서 달리고 있다. (Score: 0.7968)
원숭이 한 마리가 드럼을 연주한다. (Score: 0.2194)
한 남자가 담으로 싸인 땅에서 백마를 타고 있다. (Score: 0.1424)
한 남자가 음식을 먹는다. (Score: 0.1308)
한 남자가 빵 한 조각을 먹는다. (Score: 0.0911)


In [42]:
import pandas as pd

df = pd.read_csv('./data/train.csv')

df

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
0,TRAIN_000,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,건축구조,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,TRAIN_001,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,건축구조,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있..."
2,TRAIN_002,철골구조의 장점이 뭐야?,철골구조의 장점을 알려줘?,건축구조,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 때문에 고층 건물의 건축이 용이...,"철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고, 고층 건...",철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...
3,TRAIN_003,철골철근 콘크리트 구조가 뭐야?,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,건축구조,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...",철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 일반적으로 사용되는 방식...,"철골철근콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건...","철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ..."
4,TRAIN_004,철골구조는 어떤 방식이 있어?,철골구조의 다양한 방식이 무엇인가요?,건축구조,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다. 이들은 건축 시스템에 따...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다. 이외에도 최근에는 고층 건...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...
...,...,...,...,...,...,...,...,...,...
639,TRAIN_639,벽장 부위 결로의 원인이 뭐야?,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,타 마감하자,벽장 부위 결로의 원인은 난방이 이웃한 방과동일한 조건이 되나 그 방에 비해 저온인...,"벽장 부위 결로의 원인은 주로 난방이 자리잡은 방이 내부 온도가 낮은 반면, 외부 ...",벽장 부위 결로가 발생하는 원인은 난방이 잘 이루어지지 않아 해당 공간이 저온인 반...,"벽장 부위 결로가 발생하는 원인은 난방이 인접한 방과 같은 조건을 갖추고 있지만, ...",벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
640,TRAIN_640,"AD, PD에 면한 벽체 결로의 원인이 뭐야?",벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,타 마감하자,"AD, PD에 면한 벽체 결로의 원인은 외기에 접하는 면 좌, 우측 벽체에는 단열재...","AD, PD에 면한 벽체 결로의 원인은 외기에 접하는 면 좌, 우측 벽체에는 단열재...","AD, PD에 면한 벽체 결로의 주된 원인은 외기에 접하는 면 좌, 우측 벽체에 단...","AD, PD에 면한 벽체 결로의 원인으로는 외기에 접하는 면 좌, 우측 벽체는 일반...","AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각..."
641,TRAIN_641,외벽 모서리 부위에 결로가 발생하는 원인이 뭐야?,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,타 마감하자,외벽모서리 부위에 결로가 발생하는 원인은 높은 온도차 때문입니다. 외벽 모서리의 경...,외벽 모서리 부위에 결로가 발생하는 원인은 중요한 역할을 합니다. 외부에 비치는 세...,외벽 모서리 부위에 결로가 발생하는 원인으로는 높은 온도차가 대표적입니다. 외벽 모...,외벽 모서리 부위에 결로가 발생하는 원인은 실내와 외부의 온도차 때문입니다. 외벽 ...,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
642,TRAIN_642,창호 결로의 대책은 뭐야?,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,타 마감하자,창호결로의 대책은 제품을 선정할 때 KS에 규정된 프레임을 선정하고 유리의 열관류율...,창호 결로의 대책은 KS에 규정된 프레임을 선정하고 유리의 열관류율 및 결로 발생 ...,"창호 결로를 예방하기 위한 대책은 KS에 규정된 프레임을 선택하고, 열관류율이 적은...","창호 결로의 대책은 여러 가지가 있습니다. 먼저, 창호를 선택할 때 KS에 규정된 ...","창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및..."


In [43]:
values_list = []
for column in ['질문_1', '질문_2', '답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
    values_list.extend(df[column].tolist())

len(values_list), values_list[:20] 

(4508,
 ['면진장치가 뭐야?',
  '내진설계의 종류 좀 알려줘',
  '철골구조의 장점이 뭐야?',
  '철골철근 콘크리트 구조가 뭐야?',
  '철골구조는 어떤 방식이 있어?',
  '커튼월이 뭐야?',
  '내진구조가 뭐야?',
  '중목구조 방식이 뭐야?',
  '기둥-보 구조 방식이 뭐야?',
  '블록구조가 뭐야',
  '철골구조의 단점이 뭐야?',
  '콘크리트 구조는 어떤 방식이 있어?',
  '철골콘크리트 구조가 뭐야?',
  '면진구조가 뭐야?',
  '철골구조가 뭐야?',
  '면진구조와 제진구조의 차이점이 뭐야?',
  '철근콘크리트 구조가 뭐야?',
  '건축물의 구조는 어떻게 구분해?',
  '조적식 구조는 어떤 방식이 있어?',
  '목구조는 어떤 방식이 있어?'])

In [44]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
from sentence_transformers import SentenceTransformer, util
import numpy as np

embedder = SentenceTransformer("jhgan/ko-sroberta-multitask")

corpus_embeddings = embedder.encode(values_list, convert_to_tensor=True)

# Query sentences:
queries = ['토목구조는 어떤 방식이 있어?',
  '블록구조의 단점이 뭔가요?',
  '내진설계의 장점으로는 뭐가 있을까요?',
  '면진장치가 뭐야?']

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
for query in queries:
 query_embedding = embedder.encode(query, convert_to_tensor=True)
 cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
 cos_scores = cos_scores.cpu()

 #We use np.argpartition, to only partially sort the top_k results
 top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

 print("\n\n======================\n\n")
 print("Query:", query)
 print("\nTop 5 most similar sentences in corpus:")

 for idx in top_results[0:top_k]:
  print(values_list[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))
embedder = SentenceTransformer()





Query: 토목구조는 어떤 방식이 있어?

Top 5 most similar sentences in corpus:
토목이란 무엇인가요? (Score: 0.8570)
토목이 뭐야? (Score: 0.8440)
콘크리트 구조는 어떤 방식이 있어? (Score: 0.7799)
방수공사 공법의 종류는 뭐가 있어? (Score: 0.7405)
토목이란 목재, 철재, 토석 등을 써서 도로, 교량, 항만, 제방, 철도, 건물, 상하수도 등을 건설하는 공사를 의미합니다. (Score: 0.7328)




Query: 블록구조의 단점이 뭔가요?

Top 5 most similar sentences in corpus:
철골구조의 단점이 뭐야? (Score: 0.7763)
노출콘크리트의 단점은 무엇인가요? (Score: 0.7297)
노출콘크리트의 단점이 뭐야? (Score: 0.7229)
큐블럭의 단점이 뭐야? (Score: 0.7072)
블록구조가 뭐야 (Score: 0.6926)




Query: 내진설계의 장점으로는 뭐가 있을까요?

Top 5 most similar sentences in corpus:
내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요? (Score: 0.7864)
내진설계의 종류 좀 알려줘 (Score: 0.7460)
내진설계에서 안전성을 높이기 위한 순서는 무엇인가요? (Score: 0.7210)
내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다. (Score: 0.6974)
내진구조가 뭐야? (Score: 0.6928)




Query: 면진장치가 뭐야?

Top 5 most similar sentences in corpus:
면진장치가 뭐야? (Score: 1.0000)
면진장치에 사용되는 주요 기술은 무엇인가요? (Score: 0.8199)
못, 또는 기타 이물질로 인해 면 불량이 발생할 수 있어? (Score: 0.6619)
면 불량은 뭐야? (Score: 0.6536)
면에 이물질이나 손상이 발생할 수 있는 